In [ ]:
import os
import cv2
import pandas as pd
from skimage import feature
import numpy as np


In [ ]:
diretorio_imagens = './data/patches/'
diretorio_caracteristicas = './data/caracteristicas/'
arquivo_csv = './data/caracteristicas/LBP.csv'

# Se o diretório não existir, cria
if not os.path.exists(diretorio_caracteristicas):
    os.makedirs(diretorio_caracteristicas)


In [ ]:
def calcular_LBP(imagem):
    lbp = feature.local_binary_pattern(imagem, 8, 1, method="uniform")
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 59))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)
    return hist


In [ ]:
# Inicialize um DataFrame vazio para armazenar as características LBP
df = pd.DataFrame()

# Percorra todos os diretórios [key]
for key in os.listdir(diretorio_imagens):
    key_path = os.path.join(diretorio_imagens, key)
    
    # Percorra todas as imagens no diretório [key]
    for image_name in os.listdir(key_path):
        image_path = os.path.join(key_path, image_name)
        
        # Carregue a imagem
        imagem = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        
        # Calcule as características LBP
        caracteristicas_lbp = calcular_LBP(imagem)

        # Obtenha o nome do arquivo original
        nome_arquivo_original = image_name.split('_')[0]
        
        # Adicione as características e o nome do arquivo ao DataFrame
        df = df.append(pd.Series([key, nome_arquivo_original] + list(caracteristicas_lbp)), ignore_index=True)

        print(f'Processando {image_path}')

# Defina os nomes das colunas no DataFrame
colunas = ['key', 'nome_arquivo'] + [f'LBP_{i}' for i in range(len(caracteristicas_lbp))]
df.columns = colunas

# Salve o DataFrame em um arquivo CSV
df.to_csv(arquivo_csv, index=False)
